In [1]:
# math stuff
import numpy as np

# tensorflow all the things
import tensorflow as tf

# suppress numpy scientific e-notation for better readability
np.set_printoptions(suppress=True)

In [2]:
# tensorflow version
print(tf.__version__)

1.4.1


In [3]:
# create dummy data similar to what my database query returns (several features over time)
smooth_data = np.float64(np.arange(4000).reshape((1000, 4)))
print(smooth_data)

[[    0.     1.     2.     3.]
 [    4.     5.     6.     7.]
 [    8.     9.    10.    11.]
 ..., 
 [ 3988.  3989.  3990.  3991.]
 [ 3992.  3993.  3994.  3995.]
 [ 3996.  3997.  3998.  3999.]]


In [4]:
# set generator input, parameters, tensorflow sos, eos and pad tokens

# create variables and assign values
sequence = smooth_data
observation_length = 4 # data the network will see
prediction_length = 2 # data the network will learn to predict
stride = 2 # number of time steps the window will incrementally slide for each batch 
total_length = observation_length + prediction_length # total length of observation and prediction array

# define generator function
def gen_seq():
    
    # compute number of batches to emit
    num_of_batches = round(((len(sequence) - total_length) / stride))
    
    # transform and emit data in batches
    for i in range(0, num_of_batches * stride, stride):
        result = np.array(sequence[i:i + total_length])
        
        # flip array upside down as data is ordered by date desc
        result_flipped = np.flipud(result)
        
        # preparing encoder inputs, adding end of sequence token
        enc_inp = result_flipped[0:observation_length]
        
        # preparing decoder inputs, adding go and end of sequence tokens
        dec_inp = result_flipped[observation_length:total_length]
        dec_inp = dec_inp[:,0].reshape((prediction_length, 1))
        
        # preparing target values, adding end of sequence tokens
        dec_exp_out = result_flipped[observation_length:total_length]
        dec_exp_out = dec_exp_out[:,0].reshape((prediction_length, 1))
        
        # yield results
        yield enc_inp, dec_inp, dec_exp_out

gen = gen_seq()

In [5]:
enc_inp, dec_inp, dec_exp_out = next(gen)
print(enc_inp)
print("")
print(dec_inp)
print("")
print(dec_exp_out)
print("")

[[ 20.  21.  22.  23.]
 [ 16.  17.  18.  19.]
 [ 12.  13.  14.  15.]
 [  8.   9.  10.  11.]]

[[ 4.]
 [ 0.]]

[[ 4.]
 [ 0.]]



In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# batch-major vs time-major tensors 
# time-major is computationally less expensive
time_major = True

# number of features going into the encoder
dim_width_enc_inp = len(smooth_data[1])

# number of features going into the decoder that match the target sequence
dim_width_dec_inp = 1
dim_width_dec_exp_out = dim_width_dec_inp

# number of batches used in each iteration for parallel processing
batch_size = 5

# length of input and output
# [batch_size] vector for time-major = [5, 4, 6] for three sequences with lengths 5, 4 and 6
seq_length_inp = tf.fill([batch_size], observation_length)
seq_length_out = tf.fill([batch_size], prediction_length)

# create tensorflow placeholder
enc_inp = tf.placeholder(
    tf.float32, 
    shape=(observation_length, batch_size, dim_width_enc_inp), 
    name="encoder_input")
dec_inp = tf.placeholder(
    tf.float32, 
    shape=(prediction_length, batch_size, dim_width_dec_inp), 
    name="decoder_input")
dec_exp_out = tf.placeholder(
    tf.float32, 
    shape=(prediction_length, batch_size, dim_width_dec_exp_out), 
    name="target_sequence")

# create tensorflow dataset from generator
dataset = tf.data.Dataset.from_generator(
    gen_seq,
    (tf.float32, 
     tf.float32,
     tf.float32),
    (tf.TensorShape([observation_length, dim_width_enc_inp]), 
     tf.TensorShape([prediction_length, dim_width_dec_inp]),
     tf.TensorShape([prediction_length, dim_width_dec_exp_out]))
    )

# prefetch 'batch_size' generator outputs
prefetched = dataset.prefetch(batch_size)

# batch prefetched together
batches = prefetched.batch(batch_size)

# create one shot iterator
enc_inp, dec_inp, dec_exp_out = batches.make_one_shot_iterator().get_next()

# convert input to tensors and transpose for time major
enc_inp, dec_inp, dec_exp_out = sess.run([enc_inp, dec_inp, dec_exp_out])

enc_inp = tf.convert_to_tensor(enc_inp, dtype=tf.float32)
enc_inp = tf.transpose(enc_inp, perm = [1, 0, 2])
dec_inp = tf.convert_to_tensor(dec_inp, dtype=tf.float32)
dec_inp = tf.transpose(dec_inp, perm = [1, 0, 2])
dec_exp_out = tf.convert_to_tensor(dec_exp_out, dtype=tf.float32)
dec_exp_out = tf.transpose(dec_exp_out, perm = [1, 0, 2])

# defining layers and number of units for basic lstm cells
enc_layers = 2
enc_num_units = 3
dec_layers = enc_layers
dec_num_units = enc_num_units

enc_cells = []
for i in range(enc_layers):
    with tf.variable_scope("rnn_enc") as rnn_enc:
        enc_cells.append(tf.nn.rnn_cell.BasicLSTMCell(enc_num_units))
enc_cell = tf.nn.rnn_cell.MultiRNNCell(enc_cells)

dec_cells = []
for i in range(dec_layers):
    with tf.variable_scope("rnn_dec") as rnn_dec:
        dec_cells.append(tf.nn.rnn_cell.BasicLSTMCell(dec_num_units))
dec_cell = tf.nn.rnn_cell.MultiRNNCell(dec_cells)

# encoder
enc_out, enc_state = tf.nn.dynamic_rnn(
    enc_cell, 
    enc_inp,
    dtype = tf.float32,
    sequence_length = seq_length_inp,
    time_major = time_major)

# helper
helper = tf.contrib.seq2seq.TrainingHelper(
    dec_inp, 
    seq_length_out, 
    time_major = time_major)

# decoder
decoder = tf.contrib.seq2seq.BasicDecoder(
    dec_cell, 
    helper, 
    enc_state)

# dynamic decoding
dec_out, dec_state, dec_out_seq_length = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major = time_major)

# create variables for weights and biases
weights = tf.Variable(tf.random_normal([dim_width_dec_inp, enc_num_units]))
biases = tf.Variable(tf.random_normal([enc_num_units], mean=1.0))

# initialize global variables / check for uninitialized variables
sess.run(tf.global_variables_initializer())
#print(sess.run(tf.report_uninitialized_variables()))

# extract logits from decoder output
logits = dec_out.rnn_output

print("encoder input [max_time, batch_size, num_features]")
print(enc_inp)
print("")
print("decoder input [max_time, batch_size, num_features]")
print(dec_inp)
print("")
print("decoder expected output [max_time, batch_size, num_features]")
print(dec_exp_out)
print("")
print("encoder output [max_time, batch_size, num_units]")
print(enc_out)
print("")
print("encoder state [batch_size, num_units]")
print(enc_state)
print("")
print("final state [batch_size, num_units]")
print(dec_state)
print("")
print("logits [max_time, batch_size, num_units]")
print(logits.eval())

encoder input [max_time, batch_size, num_features]
Tensor("transpose:0", shape=(4, 5, 4), dtype=float32)

decoder input [max_time, batch_size, num_features]
Tensor("transpose_1:0", shape=(2, 5, 1), dtype=float32)

decoder expected output [max_time, batch_size, num_features]
Tensor("transpose_2:0", shape=(2, 5, 1), dtype=float32)

encoder output [max_time, batch_size, num_units]
Tensor("rnn/TensorArrayStack/TensorArrayGatherV3:0", shape=(4, 5, 3), dtype=float32)

encoder state [batch_size, num_units]
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(5, 3) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(5, 3) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(5, 3) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(5, 3) dtype=float32>))

final state [batch_size, num_units]
(LSTMStateTuple(c=<tf.Tensor 'decoder/while/Exit_3:0' shape=(5, 3) dtype=float32>, h=<tf.Tensor 'decoder/while/Exit_4:0' shape=(5, 3) dtype=float32>), LSTMStateTuple(c=<tf.T